In [2]:
import PIL,torch
import matplotlib.pyplot as plt
import torch.nn as nn
import numpy as np
import os
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset,DataLoader
from efficientnet_pytorch import EfficientNet

In [3]:
class SkinDataset(Dataset):
    folds=5
    def __init__(self, root , num_classes,fold=0,training=False,transform=None):
        self.data_path = []
        self.transform = transform
        self.sides = 224
        if self.transform is None:
            self.transform = transforms.Compose([
                transforms.Resize((self.sides,self.sides)),
                transforms.ToTensor(),
                transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
            ])
        self.training = training
        for label in range(num_classes):
            self.data_dir = os.path.join(root,os.listdir(root)[label])
            self.filename = os.listdir(self.data_dir)
            l = len(self.filename)
            inter = l//SkinDataset.folds
            picked = list(range(inter* fold,inter * (fold+1))) if not training else list(range(0,inter*fold))+list(range(inter*(fold+1),l))

            for i in picked:
                file_path = os.path.join(self.data_dir , self.filename[i])
#                 img = Image.open(fil)
                self.data_path.append((file_path, label))
    
    def __getitem__(self , index):
        ddir , label = self.data_path[index]
        img = Image.open(ddir)
        imgmat = self.transform(img)
#         if self.training:
#             imgmat += torch.randn(3,self.sides,self.sides)/256
        result = (imgmat, label)
        del imgmat
        del img
        return result
    
    def __len__(self):
        return len(self.data_path)

In [4]:
# del training_dataloader
batch_size = 64
training_dataset = SkinDataset(os.path.join(os.getcwd(),'../Skin40'), 40, training=True)
test_dataset = SkinDataset(os.path.join(os.getcwd(),'../Skin40'), 40)
training_dataloader = DataLoader(training_dataset , batch_size = batch_size , num_workers = 2, shuffle = True)
test_dataloader = DataLoader(test_dataset , batch_size = batch_size , num_workers = 2, shuffle = True)

In [10]:
def train(model, train_loader, loss_func, optimizer, device):
    """
    train model using loss_fn and optimizer in an epoch.
    model: CNN networks
    train_loader: a Dataloader object with training data
    loss_func: loss function
    device: train on cpu or gpu device
    """
    total_loss = 0
    model.train()
    # train the model using minibatch
    for i, (images, targets) in enumerate(train_loader):
        images = images.to(device)
        targets = targets.to(device)
        
        # forward
        outputs = model(images)
        loss = loss_func(outputs, targets)

        # backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
        # every 100 iteration, print loss
        if (i + 1) % 6 == 0:
            print ("Step [{}/{}] Train Loss: {:.4f}"
                   .format(i+1, len(train_loader), loss.item()))
    return total_loss / len(train_loader)

In [11]:
def evaluate(model, val_loader, device, nclasses):
    """
    model: CNN networks
    val_loader: a Dataloader object with validation data
    device: evaluate on cpu or gpu device
    return classification accuracy of the model on val dataset
    """
    # evaluate the model
    model.eval()
    # context-manager that disabled gradient computation
    with torch.no_grad():
        correct = 0
        total = 0
        cm = torch.zeros(nclasses, nclasses)
        for i, (images, targets) in enumerate(val_loader):
            # device: cpu or gpu
            images = images.to(device)
            targets = targets.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, dim=1)
            
            
            for t, p in zip(targets.view(-1), predicted.view(-1)):
                cm[t.long()][p.long()] +=1
            correct += (predicted == targets).sum().item()
            total += targets.size(0)
            
        accuracy = correct / total
        print('Accuracy on Test Set: {:.4f} %'.format(100 * accuracy))
        show_heatmap(cm, nclasses)
        return accuracy

In [12]:
import matplotlib.pyplot as plt
def show_curve(ys, title):
    """
    plot curlve for Loss and Accuacy
    Args:
        ys: loss or acc list
        title: loss or accuracy
    """
    x = np.array(range(len(ys)))
    y = np.array(ys)
    plt.plot(x, y, c='b')
    plt.axis()
    plt.title('{} curve'.format(title))
    plt.xlabel('epoch')
    plt.ylabel('{}'.format(title))
    plt.show()

def show_heatmap(cm, nclasses):
    labels = range(nclasses)
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111)
    ax.set_yticks(labels)
    ax.set_yticklabels(labels)
    ax.set_xticks(labels)
    ax.set_xticklabels(labels)
    
    im = ax.imshow(cm, cmap=plt.cm.hot_r )
    plt.colorbar(im)
    plt.title("Confusioin Matrix")
    plt.show()
    print('Accuracy of each class:\n', cm.diag()/(cm.sum(1)+1e-6))
    print('Recall of each class:\n', cm.diag()/(cm.sum(0)+1e-6))

In [13]:
def fit(model, num_epochs, optimizer, schedulr, device,nclasses):
    """
     train and evaluate an classifier num_epochs times.
    n and evaluate an classifier num_epochs times.
    We use optimizer and cross entropy loss to train the model. 
    Args: 
        model: CNN network
        num_epochs: the number of training epochs
        optimizer: optimize the loss function    loss_func.to(device)
    loss_func.to(device)
        schedulr: scheduling learning rate

    """
    # loss and optimizer
    loss_func = nn.CrossEntropyLoss()
    
    model.to(device)
    loss_func.to(device)
    
    # log train loss and test accuracy
    losses = []
    accs = []
    
    for epoch in range(num_epochs):
        
        print('Epoch {}/{}: , lr = {}'.format(epoch + 1, num_epochs , optimizer.param_groups[0]['lr']))
        # train step
        loss = train(model, training_dataloader, loss_func, optimizer, device)
        losses.append(loss)
        schedulr.step()
        
        # evaluate step
        accuracy = evaluate(model, test_dataloader, device,nclasses)
        accs.append(accuracy)
        
    
    # show curve
    show_curve(losses, "train loss")
    show_curve(accs, "test accuracy")

In [16]:
net = EfficientNet.from_pretrained('efficientnet-b7')
net._fc = torch.nn.Sequential(nn.Linear(2560, 4096),
                             nn.ReLU(inplace=True),
                             nn.Dropout(p=0.6),
                             nn.Linear(4096, 4096),
                             nn.ReLU(inplace=True),
                             nn.Dropout(p=0.5),
                             nn.Linear(4096, 40)
                            )

Loaded pretrained weights for efficientnet-b7


In [8]:
net._fc

Linear(in_features=2560, out_features=1000, bias=True)

In [24]:
# Hyper-parameters
num_epochs = 50
lr =1e-4
nclasses = 40
feature_tune=True
# Device configuration
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")


for parma in net.parameters():
    parma.requires_grad = feature_tune
for param in net._fc.parameters():
    param.requires_grad = True
# # optimizer
# optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, net.parameters()), lr=lr)
optimizer = torch.optim.SGD(filter(lambda p: p.requires_grad, net.parameters()), lr=lr)
schedulr = torch.optim.lr_scheduler.StepLR(optimizer , step_size = 2 , gamma = 0.83)
for name,param in net.named_parameters():
    print(param.requires_grad, name)

True _conv_stem.weight
True _bn0.weight
True _bn0.bias
True _blocks.0._depthwise_conv.weight
True _blocks.0._bn1.weight
True _blocks.0._bn1.bias
True _blocks.0._se_reduce.weight
True _blocks.0._se_reduce.bias
True _blocks.0._se_expand.weight
True _blocks.0._se_expand.bias
True _blocks.0._project_conv.weight
True _blocks.0._bn2.weight
True _blocks.0._bn2.bias
True _blocks.1._depthwise_conv.weight
True _blocks.1._bn1.weight
True _blocks.1._bn1.bias
True _blocks.1._se_reduce.weight
True _blocks.1._se_reduce.bias
True _blocks.1._se_expand.weight
True _blocks.1._se_expand.bias
True _blocks.1._project_conv.weight
True _blocks.1._bn2.weight
True _blocks.1._bn2.bias
True _blocks.2._depthwise_conv.weight
True _blocks.2._bn1.weight
True _blocks.2._bn1.bias
True _blocks.2._se_reduce.weight
True _blocks.2._se_reduce.bias
True _blocks.2._se_expand.weight
True _blocks.2._se_expand.bias
True _blocks.2._project_conv.weight
True _blocks.2._bn2.weight
True _blocks.2._bn2.bias
True _blocks.3._depthwise_c

In [26]:
fit(net, num_epochs, optimizer, schedulr,device, nclasses)  #

Epoch 1/50: , lr = 0.0001


RuntimeError: CUDA out of memory. Tried to allocate 38.00 MiB (GPU 1; 31.75 GiB total capacity; 18.24 GiB already allocated; 35.19 MiB free; 18.42 GiB reserved in total by PyTorch)